In this project, you will begin working with COBRA (COnstraint-Based Reconstruction and Analysis) methods using the Python library [cobrapy](https://opencobra.github.io/cobrapy/). Originally implemented in MATLAB as the COBRA Toolbox, cobrapy is an object-oriented programming framework for the reconstruction and analysis of constraint-based metabolic models. Although still not as comprehensive as the COBRA Toolbox when it comes to modeling capabilities and implemented algorithms, it provides ample functionality to cover our needs for this course. Before starting with this project, I strongly encourage you to take a look at the [documentation](https://cobrapy.readthedocs.io/en/latest) and [API reference](https://cobrapy.readthedocs.io/en/latest/autoapi/index.html) where you will find useful descriptions and minor tutorials on how to work with the library. We will also be using the web-based tool [Escher](https://escher.github.io/#/}) for the visualization of metabolic pathway maps and calculated flux phenotypes.


In [ ]:
# Imports:
import cobra as c


### 1.1 Toy model

Consider the following toy metabolic network with 8 reactions ($\mathrm{R}_1$, ..., $\mathrm{R}_8$) and 8 metabolites ($\mathrm{A}$, $\mathrm{B}$, $\mathrm{C}$, $\mathrm{D}$, $\mathrm{E}$, $\mathrm{ATP}$, $\mathrm{ADP}$, and $\mathrm{P}_\mathrm{i}$) (see supplementary pdf file).

(i) Specify which metabolites are transported across the cell boundaries (e.g., cell membrane), and the direction of transport.

(ii) Write down the stoichiometric matrix **S** using the ordering of reactions and metabolites as defined above. How many degrees of freedom does this reaction system have and what is the dimensionality of the solution space (i.e. null space of **S**)?

(iii) Given an upper flux bound for $\mathrm{R}_1$ of 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, what is the maximal attainable flux through reaction $\mathrm{R}_8$ and the corresponding flux distribution? What is the net production of ATP (i.e., the flux through $\mathrm{R}_7$)? Implement the model using cobrapy and verify your answer by selecting $\mathrm{R}_8$ as the objective and maximizing its flux.

The `objective` is an attribute of the Model object, while the lower and upper flux bounds of a reaction is given by the attributes `lower_bound` and `upper_bound` of the corresponding Reaction object, respectively.

In [1]:
### Define model: ---
model = c.Model("Toy model")

### Metabolites: ---
A = c.Metabolite("A")
B = c.Metabolite("B")
C = c.Metabolite("C")
D = c.Metabolite("D")
E = c.Metabolite("E")
ATP = c.Metabolite("ATP")
ADP = c.Metabolite("ADP")
P_i = c.Metabolite("P_i")

### Reactions: ---
R1 = c.Reaction("R1")
R1.add_metabolites({
   None : -1.0,
   A : 1.0,
})

R2 = c.Reaction("R2")
R2.add_metabolites({
   A : -1.0,
   ADP : -1.0,
   P_i : -1.0,
   B : 1.0,
   C : 1.0,
   ATP : 1.0
})

R3 = c.Reaction("R3")
R3.add_metabolites({
   C : -1.0,
   ATP : -1.0,
   B : 1.0,
   ADP : 1.0,
   P_i : 1.0
})

R4 = c.Reaction("R4")
R4.add_metabolites({
   B : -1.0,
   C : 1.0
})

R5 = c.Reaction("R5")
R5.add_metabolites({
   A : -1.0,
   D : -1.0,
   ADP : -1.0,
   P_i : -1.0,
   E : 1.0,
   ATP : 1.0
})

R6 = c.Reaction("R6")
R6.add_metabolites({
   E : -1.0,
   C : 1.0
})

R7 = c.Reaction("R7")
R7.add_metabolites({
   ATP : -1.0,
   ADP : 1.0,
   P_i : 1.0
})

R8 = c.Reaction("R8")
R8.add_metabolites({
   D : -1.0,
   None : 1.0
})


### Add reactions to model: ---
model.add_reactions([R1, R2, R3, R4, R5, R6, R7, R8])

for reaction in model.reactions:
    print(f"{reaction.id} : {reaction.reaction}")

NameError: name 'c' is not defined

(iv) It has been shown that the maximization of ATP yield in certain instances is a realistic cellular objective. Given the same flux bound for $\mathrm{R}_1$ as in (iii), explain and discuss the maximal feasible net production of ATP. Verify your answer using cobrapy.

(v) Assume that the flux of reaction $\mathrm{R}_6$ is to be constrained to zero. You may implement this in the stoichiometric matrix by adding a new row to **S** where all column entries are zero except in column 6, which is 1. Explain why this will constrain the flux of reaction $\mathrm{R}_6$ to zero. What is the dimensionality of this new stoichiometric matrix?

### 1.2 *Escherichia coli* core model

(i) Download `ecoli_core_model` from Blackboard and read the model using the `read_sbml_model` function in cobrapy. Give a description of its content (i.e., number of reactions, metabolites, genes, etc.). Which metabolic subsystems are implemented in the model?

Hint: the subsystems are found in the Model attribute `groups`.

(ii) Simulate the optimal growth phenotype on aerobic, minimal glucose media, by setting the lower bound of glucose uptake ('EX_glc__D_e') to a biologically reasonable uptake rate of -18.5 mmol gDW<sup>-1</sup> h<sup>-1</sup> and the oxygen uptake EX_o2_e to -1000 mmol gDW<sup>-1</sup> h<sup>-1</sup> <sup>i</sup>. What is the maximal specific growth rate and what are the uptake fluxes of glucose, ammonia, oxygen, and inorganic phosphate in the optimal solution?

<sup>i</sup> Note that uptake reaction flux bounds by default are negative, which is due to how these traditionally are defined in constraint-based models of metabolism. A boundary metabolite X is taken up by the system using the following format for the exchange reaction X $\Longleftrightarrow$, where a positive and negative flux denotes secretion and uptake, respectively.

(iii) What is the secretion profile of anaerobic growth on glucose? Compare with that of aerobic growth on glucose (ii) and give a biochemical explanation for their differences.

(iv) Visualize the reaction fluxes of both the aerobic and anaerobic flux phenotypes using the *E. coli* core model pathway map (found [here](https://escher.github.io/#/app?map=e_coli_core.Core\%20metabolism&tool=Builder&model=e_coli_core)) by creating a dictionary of reaction ids and corresponding fluxes, then writing this to a json file. Import the data into Escher by clicking Data $\rightarrow$ Load reaction data, then select your json file. Describe and discuss the difference in flux distributions.

(v) Setting the maximal substrate uptake flux to 10 mmol gDW<sup>-1</sup> h<sup>-1</sup>, maximize growth using each of the carbon sources listed in Table 1 individually under both aerobic and anaerobic conditions.

### Table 1 
| Substrate | Exchange reaction ID |
| --- | --- |
| acetate | EX_ac_e |
| acetaldehyde | EX_acald_e |
| 2-oxoglutarate | EX_akg_e | 
| ethanol | EX_etoh_e |
| D-fructose | EX_fru_e |
| fumarate | EX_fum_e |
| D-glucose | EX_glc__D_e |
| L-glutamine | EX_gln_L_e |
| L-glutamate | EX_glu_L_e |
| D-lactate | EX_lac_D_e |
| L-malate | EX_mal_L_e |
| pyruvate | EX_pyr_e |
| succinate | EX_succ_e |